<a href="https://colab.research.google.com/github/Aayush360/Shamshad_Ansari_CV_ANN/blob/master/YOLOv3_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installing the darknet framework

In [1]:
%%shell
git clone https://github.com/ansarisam/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 5912, done.
remote: Total 5912 (delta 0), reused 0 (delta 0), pack-reused 5912
Receiving objects: 100% (5912/5912), 6.34 MiB | 2.91 MiB/s, done.
Resolving deltas: 100% (3920/3920), done.


In [3]:
# running the make command to run darknet

In [ ]:
%%shell
cd darknet/
make

In [5]:
# testing the darknet installation

In [3]:
%%shell
cd darknet 
./darknet

usage: ./darknet <function>


In [7]:
# downloading pre-trained Darknet-53 weights (of COCO datasets)

In [5]:
%%shell
mkdir pretrained
cd pretrained
wget https://pjreddie.com/media/files/darknet53.conv.74

mkdir: cannot create directory ‘pretrained’: File exists
--2020-08-24 07:50:40--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M   172KB/s    in 16m 58s 

2020-08-24 08:07:40 (156 KB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



In [10]:
# downloading the petdaset images and annotations

In [ ]:
%%shell

mkdir petdata
cd petdata
wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

tar -xvf images.tar.gz
tar -xvf annotations.tar.gz

In [14]:
# deleting the invalid file extension

In [11]:
%%shell
cd /content/petdata/images
rm *.mat

In [16]:
# converting image annotations from xml to txt

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path, img_path, label_path):
  if not os.path.exists(label_path):
    os.makedirs(label_path)
  
  class_list = []
  for xml_file in glob.glob(path + '/*.xml'):
    xml_list = []
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for member in root.findall('object'):
      imagename = str(root.find('filename').text)
      print('image ', imagename)
      index = int(imagename.rfind('_'))
      print('index', index)
      classname = imagename[0:index]

      class_index = 0

      if (class_list.count(classname) >0):
        class_index = class_list.index(classname)
      else:
        class_list.append(classname)
        class_index = class_list.index(classname)
      

      print('height: ', root.find('size').find('height').text)
      print('width: ', root.find('size').find('width').text)
      print('minx ', member[4][0].text)
      print('miny ',member[4][1].text)
      print('maxx ', member[4][2].text)
      print('maxy ', member[4][3].text)

      w = float(root.find('size').find('width').text)
      h = float(root.find('size').find('height').text)
      dw = 1.0/w
      dh = 1.0/h

      x = (float(member[4][0].text)+float(member[4][2].text))/ 2.0 -1
      y = (float(member[4][1].text)+float(member[4][3].text))/ 2.0 -1
      w = float(member[4][2].text)-float(member[4][0].text)
      h = float(member[4][3].text)-float(member[4][1].text)

      x = x*dw
      w = w*dw
      y = y* dh
      h = h*dh
      value = (class_index, x,y,y,h)
      print('The line value is : ', value)
      print('csv filename is: ', os.path.join(label_path, imagename.rsplit('.',1)[0]+'.txt'))
      xml_list.append(value)

      df = pd.DataFrame(xml_list)
      df.to_csv( os.path.join(label_path, imagename.rsplit('.',1)[0]+'.txt'), index=None, header=False, sep=' ')
  
  class_df = pd.DataFrame(class_list)
  return class_df

def create_training_and_test(image_dir, label_dir):
  file_list = []
  for img in glob.glob(image_dir +'/*'):
    print(os.path.abspath(img))
    imagefile = os.path.basename(img)
    textfile = imagefile.rsplit('.',1)[0]+'.txt'
    if not os.path.isfile(label_dir+'/'+textfile):
      print('delete image file ', img)
      os.remove(img)
      continue
    file_list.append(os.path.abspath(img))
  

  file_df = pd.DataFrame(file_list)
  train = file_df.sample(frac=0.7, random_state=10)
  test = file_df.drop(train.index)

  train.to_csv('petdata/train.txt', index=None, header=False)
  test.to_csv('petdata/test.txt', index=None, header= False)


def main():
  img_dir = 'petdata/images'
  label_dir = 'petdata/labels'

  xml_path = os.path.join(os.getcwd(),'petdata/annotations/xmls')
  img_path = os.path.join(os.getcwd(),img_dir)
  label_path = os.path.join(os.getcwd(),label_dir)

  class_df = xml_to_csv(xml_path, img_path, label_path)
  class_df.to_csv('petdata/class.data', index=None, header=False)
  create_training_and_test(img_dir, label_path)
  print('successfully converted xml to csv. ')


main()




In [14]:
mkdir yolov3_model

In [27]:
# configuring the training input

In [28]:
# training the yolov3 model

In [15]:
%%shell
cd darknet/
./darknet detector train cfg/pet_input.cfg cfg/yolov3-pet.cfg /content/pretrained/darknet53.conv.74


Streaming output truncated to the last 5000 lines.
Region 82 Avg IOU: 0.508734, Class: 0.288957, Obj: 0.140270, No Obj: 0.002802, .5R: 0.500000, .75R: 0.000000,  count: 2
Region 94 Avg IOU: 0.487203, Class: 0.034160, Obj: 0.287078, No Obj: 0.000511, .5R: 0.500000, .75R: 0.000000,  count: 2
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000008, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.490582, Class: 0.092953, Obj: 0.412616, No Obj: 0.003317, .5R: 0.500000, .75R: 0.000000,  count: 2
Region 94 Avg IOU: 0.514551, Class: 0.081928, Obj: 0.083967, No Obj: 0.000697, .5R: 0.500000, .75R: 0.000000,  count: 2
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000008, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.512790, Class: 0.380462, Obj: 0.614503, No Obj: 0.003711, .5R: 0.666667, .75R: 0.000000,  count: 3
Region 94 Avg IOU: 0.351696, Class: 0.096703, Obj: 0.415703, No Obj: 0.000572, .5R: 0.000000, .75R: 0.000000,  count: 1
Region 106 Avg IOU: -nan, C

CalledProcessError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')